# End of week 2 exercise

This is a senior prompt engineer who refines your prompt to a better prompt as you go along this course.
It supports all languages.

Addon:
 - Has a polished gradio UI to easily interact with.

In [1]:
# imports
import os
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr
import logging

In [3]:
# set up environment
load_dotenv(override=True)
openrouter_api_key=os.getenv("OPENROUTER_API_KEY")
openai_api_key=os.getenv("OPENAI_API_KEY")


# logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

# Example: define available providers and clients
available_providers = ["OpenRouter", "OpenAI", "Ollama"]
clients = {
    "OpenRouter": OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=openrouter_api_key,
    ),
    "OpenAI": OpenAI(api_key=openai_api_key),
    "Ollama": OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",
    ),
}
models = {
    "OpenAI": ["gpt-4o-mini", "gpt-3.5-turbo"],
    "OpenRouter": ["gpt-4o-mini", "orca-mini", "mpt-7b"],
    "Ollama": ["llama3.2", "llama3.2-70b-instruct", "deepseek-chat"]
}

# Track model selection per provider
selection_state = {provider: models[provider][0] for provider in available_providers}

In [4]:
system_prompt = """
You are a Senior Prompt Engineer.
Your role is to assist users in **writing new prompts from scratch** and refining existing prompts. You optimize them to be clearer, more specific, actionable, and high-quality for AI systems, including technical, creative, instructional, or multi-language tasks.

Core Responsibilities
1. For existing prompts:
   * Analyze the user’s original prompt.
   * Identify missing context, ambiguities, or weaknesses.
   * Improve clarity, structure, constraints, tone, and expected output.
2. For new prompts:
   * Guide the user step-by-step to create a complete, high-quality prompt.
   * Suggest role assignments, context, constraints, expected outputs, tone, and examples.
3. Support multi-language output.

Operating Rules
* Do NOT generate the solution or content requested by the user (e.g., do not write the poem, essay, code, or answer).
* You **can provide examples of how a good prompt should look**.
* Always preserve the user’s intent.
* Refined or newly created prompts should be structured, specific, and actionable.

Output Structure
Always respond using this format:

1. Prompt Diagnosis (for existing prompts)
* Explain what is missing, unclear, or weak.
* Identify assumptions or gaps.

2. Refined Prompt (Improved Version) or Prompt Draft
* Provide a fully rewritten or newly drafted prompt in markdown.
* Include role assignment, context, constraints, output format, and tone.
* Include examples or instructions if helpful.
* Do NOT include the solution to the user’s task.

3. Optional Enhancements
* Suggest additional constraints or alternative structures.
* Suggest multi-language variations if relevant.

Behavior Standards
* Be precise, structured, and professional.
* Optimize prompts for high-quality AI output.
* Support all types of tasks: creative, technical, instructional, multi-language.
* Avoid unnecessary verbosity.
* Always preserve the user’s original intent.

Multi-Language Support
* If the user specifies a language, refine or create prompts in that language.
* If no language is specified, use English.

Special Notes
* If the user greets you, respond briefly and ask them to provide a prompt or explain what they want to write.
* Never provide the solution or content of the task; only guide prompt creation and refinement.
"""

In [5]:
# Ask Function (Streaming)
def ask(client, model, message, history):
    """
    Example generator function for streaming responses from a client.
    Replace with your actual client streaming logic.
    """
    # Simulate streaming for demonstration
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        stream=True
    )
    
    for chunk in stream:
        delta = chunk.choices[0].delta.content
        if delta:
            yield delta

In [6]:
# Chat Function
def chat(message: str, history: list[dict], selected_provider: str, model_selector: str):
    client = clients.get(selected_provider)
    print(client)
    if not client:
        yield {"role": "assistant", "content": "Invalid provider selected."}
        return

    try:
        response = ask(client, model_selector, message, history)
        full_response = ""
        for chunk in response:
            full_response += chunk
            yield {"role": "assistant", "content": full_response}
    except Exception as e:
        yield {"role": "assistant", "content": f"Error: {str(e)}"}

In [7]:
# Provider Change Handler
def on_provider_change(change):
    logger.info(f'Provider changed to {change}')

    new_choices = models.get(change, [])
    default_model = new_choices[0] if new_choices else None
    selection_state[change] = default_model

    return gr.Dropdown(
        choices=new_choices,
        value=default_model,
        interactive=True,
    )

In [8]:
# Model Change Handler
def on_model_change(change, provider):
    logger.info(f'Selected model for {provider}: {change}')
    selection_state[provider] = change

In [9]:
# Helper Functions
def get_desired_value_or_first_item(value, choices):
    return value if value in choices else (choices[0] if choices else None)

In [10]:
# Build UI
with gr.Blocks(title='Senior Prompt Engineer', fill_width=True) as ui:

    with gr.Row():
        provider_selector = gr.Dropdown(
            choices=available_providers,
            value=get_desired_value_or_first_item(selection_state.get("OpenAI"), available_providers),
            label='Provider',
        )
        model_selector = gr.Dropdown(
            choices=models[provider_selector.value],
            value=get_desired_value_or_first_item(selection_state[provider_selector.value], models[provider_selector.value]),
            label='Model',
        )

    provider_selector.change(
        fn=on_provider_change, 
        inputs=provider_selector, 
        outputs=model_selector
    )
    model_selector.change(
        fn=on_model_change, 
        inputs=[model_selector, provider_selector], 
        outputs=[]
    )

    gr.ChatInterface(
        fn=chat,
        type='messages',
        chatbot=gr.Chatbot(type='messages', height='75vh', resizable=True),
        additional_inputs=[provider_selector, model_selector],
    )

# Launch UI
ui.launch()

INFO:httpx:HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
